### Voice Translation of Audio files 

Ever wanted to translate a podcast into your native language? Translating and dubbing podcasts can make content more accessible to a global audience. This guide will walk you through the process of converting an English audio file into Hindi using OpenAI APIs. We will cover transcription of the audio, translation into target script, text-to-speech conversion into target language, and benchmarking to ensure quality and accuracy.

A note on semantics used in this Cookbook regarding **Language** and **Script**. These words are generally used interchangeably, though it's important to understand the distinction given the task at hand. 
- **Language** refers to the spoken or written system of communication. For instance, Hindi and Marathi are different languages, but both use the Devanagari script. Similarly, English and French are different languages, but are written in Latin script. 
- **Script** refers to the set of characters or symbols used to write the language. For example, Serbian language traditionally written in Cyrillic Script, is also written in Latin script.


In this cookbook we will walk through steps to dub an audio podcast from English to Hindi. Overall steps are:    

1. **Transcribe** the audio file into text with Whisper 
2. **Translate** the English language text to Hindi in Devanagari script    
3. **Text-to-speech** conversion of the Devanagari script into spoken Hindi language 
4. **Translation benchmarking** (BLEU or ROUGE) 
5. **Interpret and improve** scores by adjusting prompting parameters in steps 1-3 as needed 

### Step 1: Transcribe the audio file into text with Whisper

Whisper is an open-source automatic speech recognition (ASR) system developed by OpenAI. It can transcribe audio files with high accuracy across multiple languages. 


![Text-to-speech](./images/Whisper.png)

*[Learn more about Whisper here](https://platform.openai.com/docs/guides/speech-to-text) 


In [38]:
from openai import OpenAI
client = OpenAI()

audio_file = "../gpt4o/data/keynote_recap.mp3"

audio_file= open(audio_file, "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1",  
  file=audio_file
)
transcription_english_first_pass = transcription.text
print(transcription_english_first_pass)

Welcome to our first-ever OpenAI Dev Day. Today, we are launching a new model, GPT-4 Turbo. GPT-4 Turbo supports up to 128,000 tokens of context. We have a new feature called JSON mode, which ensures that the model will respond with valid JSON. You can now call many functions at once, and it'll do better at following instructions in general. You want these models to be able to access better knowledge about the world. So do we. So we're launching retrieval in the platform. You can bring knowledge from outside documents or databases into whatever you're building. GPT-4 Turbo has knowledge about the world up to April of 2023, and we will continue to improve that over time. Dolly 3, GPT-4 Turbo with Vision, and the new Text-to-Speech model are all going into the API today. Today, we're launching a new program called Custom Models. With Custom Models, our researchers will work closely with the company to help them make a great custom model, especially for them and their use case using our t

Note that the model transcribed "Dall-e" as "Dolly". This is a common issue with similar sounding words. 

There are two techniques to correct such transcription errors: 

#### Option 1: Provide the correct transcription in the `prompt` parameter. 
You could build a "glossary" of terms that model is likely to transcribe inaccurately given the context of the text. Over time, terms can be added to the glossary improving accuracy.  

In [39]:
# Path to the source audio file  
audio_file = "../gpt4o/data/keynote_recap.mp3"

# Transcription errors can be stored in a glossary of transcription errors data structure or variable 
glossary_of_transcription_errors = "Dall-e"

# Invoke the model to get the transcription
audio_file= open(audio_file, "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
    prompt=glossary_of_transcription_errors,   
  file=audio_file
)

english_transcription_second_pass = transcription.text
print(english_transcription_second_pass)

Welcome to our first ever OpenAI Dev Day. Today we are launching a new model, GPT-4 Turbo. GPT-4 Turbo supports up to 128,000 tokens of context. We have a new feature called JSON mode, which ensures that the model will respond with valid JSON. You can now call many functions at once, and it'll do better at following instructions in general. You want these models to be able to access better knowledge about the world. So do we. So we're launching retrieval in the platform. You can bring knowledge from outside documents or databases into whatever you're building. GPT-4 Turbo has knowledge about the world up to April of 2023, and we will continue to improve that over time. Dall-e 3, GPT-4 Turbo with Vision, and the new Text-to-Speech model are all going into the API today. Today we're launching a new program called Custom Models. With Custom Models, our researchers will work closely with the company to help them make a great custom model, especially for them and their use case using our to

Now the model accurately transcribed word "Dall-e" as we provided the correct transcription in the prompt.
 

#### Option 2: Use GPT model for post-processing output 
Another popular technique to improve the quality of output and correct spelling mistakes is to use GPT model for post-processing as described below.

In [40]:
system_prompt = f"You are a helpful assistant. Your task is to correct any spelling discrepancies and grammar in the transcribed text. Only add necessary punctuation such as periods, commas, and capitalization, and use only the context provided. Some words that may be misspelled include: {glossary_of_transcription_errors}" 

response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": transcription_english_first_pass
            }
        ]
    )

transcription_english_third_pass = response.choices[0].message.content

### Step 2. Translate the English language text to Hindi in Devanagari script

Next step is to transcribe the text from the given language, to the target language script. In this case we prompt the gpt-4o model to translate the text from English (written in Latin script) to Hindi (written in Devanagari script). 

For certain new words in a language, there may not be an available translation in the target language. In such cases, we prompt the model to retain the words in original language. We can also provide examples of words to keep in the original script (E.g., English). We can store these terms that we want to keep in source language as a glossary of terms. Which can be expanded as we translate the text. 


In [42]:
glossary_of_terms_to_keep_in_original_language = "Some words to keep in English include - Turbo, OpenAI, token, GPT, Dall-e, Python"

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": f"You are an assistant that translates text from English to Hindi. User will provide content for you to translate. You may keep certain words in English for which a direct translation doesn't exist. {glossary_of_terms_to_keep_in_original_language} "},
    {"role": "user", "content": transcription.text},
  ]
)

hindi_transcription = response.choices[0].message.content

print(hindi_transcription)

हमारे पहले OpenAI Dev Day में आपका स्वागत है। आज हम एक नया मॉडल, GPT-4 Turbo लॉन्च कर रहे हैं। GPT-4 Turbo 128,000 tokens तक का कॉन्टेक्स्ट सपोर्ट करता है। हमारे पास एक नई फीचर है जिसे JSON मोड कहा जाता है, जो सुनिश्चित करता है कि मॉडल वैध JSON के साथ प्रतिक्रिया करेगा। आप अब एक साथ कई फंक्शन्स को कॉल कर सकते हैं, और यह सामान्य रूप से निर्देशों का पालन करने में बेहतर करेगा। आप चाहते हैं कि ये मॉडल बेहतर ज्ञान को एक्सेस कर सके। हम भी यही चाहते हैं। इसलिए हम प्लेटफार्म में रिट्रीवल लॉन्च कर रहे हैं। आप बाहरी दस्तावेज़ों या डेटाबेस से ज्ञान को ला सकते हैं जो भी आप बना रहे हैं उसमें। GPT-4 Turbo के पास अप्रैल 2023 तक की दुनिया की जानकारी है, और हम इसे समय के साथ सुधारते रहेंगे। Dall-e 3, Vision के साथ GPT-4 Turbo, और नया Text-to-Speech मॉडल आज से API में उपलब्ध हैं।

आज हम एक नया प्रोग्राम लॉन्च कर रहे हैं जिसे Custom Models कहा जाता है। Custom Models के साथ, हमारे शोधकर्ता कंपनी के साथ मिलकर काम करेंगे ताकि वे उनके उपयोग के मामले के लिए विशेष मॉडल बना सकें, हमारे औजारों का उपयोग करते हुए,

The transcribed text is a combination of Hindi and English languages, represented in their respective scripts Devanagari and Latin. This will generate a more natural sounding voice with the righ purnounciation of the words.

### 3. Text-to-speech conversion of the Devanagari script into spoken Hindi language

OpenAI tts model can take the translated text as input and provide the spoken language output with native sounding Hindi, intermingled with native sounding English words, where appropriate.   

![Text-to-speech](./images/Text-to-speech.png)
 

*[Learn more about tts model here](https://platform.openai.com/docs/guides/text-to-speech)

In [43]:
output_file_path = "./sounds/output.mp3"

response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=hindi_transcription,
)

response.stream_to_file(output_file_path)

/var/folders/jk/xl1ppdk94637355tx59m0kph0000gp/T/ipykernel_49771/3220431284.py:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(output_file_path)


In [30]:
from playsound import playsound

playsound(output_file_path)

### Step 4. Translation benchmarking (e.g., BLEU or ROUGE) 

Assess the quality of the translated text by comparing it to a reference translation using evaluation metrics like BLEU and ROUGE. 

**BLEU (Bilingual Evaluation Understudy)**: Measures the overlap of n-grams between the candidate and reference translations. Scores range from 0 to 100, with higher scores indicating better quality.

**ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**: Commonly used for summarization evaluation. Measures the overlap of n-grams and longest common subsequence between candidate and reference texts.

Typically, we need a reference translation (human-translated version) of the original text for accurate evaluation. However, for the purposes of this example, we will translate the text back from Hindi to English to get a metric on quality of translation. 

In [46]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are an assistant that translates text from Hindi to English. User will provide content for you to translate."},
    {"role": "user", "content": hindi_transcription},
  ]
)

english_re_transcription = response.choices[0].message.content

print(english_re_transcription)

Welcome to our first OpenAI Dev Day. Today, we are launching a new model, GPT-4 Turbo. GPT-4 Turbo supports context up to 128,000 tokens. We have a new feature called JSON mode, which ensures the model responds with valid JSON. You can now call multiple functions simultaneously, and it will generally perform better in following instructions. You want these models to access better knowledge. We want the same. So, we are launching retrieval in the platform. You can pull knowledge from external documents or databases into whatever you are building. GPT-4 Turbo has world information up to April 2023, and we will continue to improve it over time. Dall-e 3, Vision with GPT-4 Turbo, and the new Text-to-Speech model are available in the API starting today.

Today, we are launching a new program called Custom Models. With Custom Models, our researchers will work with companies to create specific models for their use cases, using our tools, with higher rate limits. We are doubling the tokens per

In [45]:
import sacrebleu
from rouge_score import rouge_scorer

# We'll use the original english transcription as the reference text 
reference_text = transcription_english_third_pass

candidate_text = english_re_transcription

# BLEU Score Evaluation
bleu = sacrebleu.corpus_bleu([candidate_text], [[reference_text]])
print(f"BLEU Score: {bleu.score}")

# ROUGE Score Evaluation
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_text, candidate_text)
print(f"ROUGE-1 Score: {scores['rouge1'].fmeasure}")
print(f"ROUGE-L Score: {scores['rougeL'].fmeasure}")


BLEU Score: 49.23275305980889
ROUGE-1 Score: 0.8346613545816735
ROUGE-L Score: 0.7669322709163345


### Step 5. Interpret and improve scores by adjusting prompting parameters in steps 1-3 as needed

In this example, both BLEU and ROUGE scores indicates we are close to an excellent outcome. 

**Interpreting BLEU Scores:** While there is no universally accepted scale, some interpretations suggest:

0 to 10: Poor quality translation; significant errors and lack of fluency.

10 to 20: Low quality; understandable in parts but contains many errors.

20 to 30: Fair quality; conveys the general meaning but lacks precision and fluency.

30 to 40: Good quality; understandable and relatively accurate with minor errors.

40 to 50: Very good quality; accurate and fluent with very few errors.

50 and above: Excellent quality; closely resembles human translation.

**Interpreting ROUGE scores:** The interpretation of what constitutes a "good" ROUGE score can vary depending on several factors, including the specific task, dataset, and domain. Following guidelines indicate a good outcome. 

ROUGE-1 (unigram overlap): Scores between 0.5 to 0.6 are generally considered good for abstractive summarization tasks.

ROUGE-L (Longest Common Subsequence): Scores around 0.4 to 0.5 are often regarded as good, reflecting the model's ability to capture the structure of the reference text.

If the score for your translation is unsatisfactory, consider the following questions: 

#### 1. Is the source audio accurately transcribed? 
Revisit parameters `glossary_of_transcription_errors` and add text that has incorrect transcriptions.  

#### 2. Is the transcribed text free of grammatical errors? 
Consider using a post-processing step with GPT model to refine the transcription to remove grammatical mistakes and add appropriate punctuations. 

#### 3. Are there words that make sense to keep in the original language?  
There may be new terms or concepts for which a translation in target language may not exist or is not universally understood. Revisit `glossary_of_terms_to_keep_in_original_language` and add such terms.

To recap this guide provides a step-by-step process for translating and dubbing audio from one language to another, making content more accessible to a global audience. The example we used is voice translate an audio file from English to Hindi. The process includes:

**1. Transcription**: Using OpenAI's Whisper to transcribe the English audio into text.

**2. Translation**: Converting the transcribed English text into Hindi, specifically in the Devanagari script.

**3. Text-to-Speech**: Transforming the translated text into spoken Hindi using a text-to-speech service.

**4. Benchmarking**: Evaluating the quality and accuracy of the translation with metrics like BLEU or ROUGE and refining the results by adjusting parameters throughout the process.

The guide also clarifies the distinction between "language" and "script," which are often used interchangeably but have specific meanings critical to the translation task. Language refers to the spoken or written system of communication, while script refers to the characters used to write the language. Understanding this distinction is key to effectively translating and dubbing content. 